In [1]:
%pylab inline
import numpy as np
import scipy as sp
import tensorflow as tf
import pandas as pd
import seaborn as sns
import itertools,time
import sys, os
from collections import OrderedDict
from copy import deepcopy
import numpy as np
from time import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.datasets import fetch_20newsgroups
import matplotlib.pyplot as plt
import itertools
import time
import pickle

Populating the interactive namespace from numpy and matplotlib


/afs/inf.ed.ac.uk/group/cup/data1/akash_itm/IPC_MAP/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
dataset_tr = '/afs/inf.ed.ac.uk/group/cup/data1/akash_itm/IPC_MAP/NTP/process_UCI/M_nips.full_docs.mat'
data_tr = sp.io.loadmat(dataset_tr)['M'].T.toarray()

vocab_ = '/afs/inf.ed.ac.uk/group/cup/data1/akash_itm/IPC_MAP/NTP/process_UCI/vocab.nips.txt'
vocab_ = open(vocab_,'r').readlines()
vocab=np.array([word for word in vocab_])

print data_tr.shape, vocab.shape

(1500, 12419) (12419,)


In [3]:
vocab_size=len(vocab)

In [4]:
n_samples_tr = data_tr.shape[0]
docs_tr = data_tr

In [5]:
def xavier_init(fan_in, fan_out, constant=1): 
    """ Xavier initialization of network weights"""
    # https://stackoverflow.com/questions/33640581/how-to-do-xavier-initialization-on-tensorflow
    low = -constant*np.sqrt(6.0/(fan_in + fan_out)) 
    high = constant*np.sqrt(6.0/(fan_in + fan_out))
    return tf.random_uniform((fan_in, fan_out), 
                             minval=low, maxval=high, 
                             dtype=tf.float32)
def log_dir_init(fan_in, fan_out,topics=50): 
    return tf.log((1.0/topics)*tf.ones([fan_in, fan_out]))

tf.reset_default_graph()
class VariationalAutoencoder(object):
    """ 
    See "Auto-Encoding Variational Bayes" by Kingma and Welling for more details.
    """
    def __init__(self, network_architecture, transfer_fct=tf.nn.softplus, 
                 learning_rate=0.001, batch_size=100):
        self.network_architecture = network_architecture
        self.transfer_fct = transfer_fct
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        print 'Learning Rate:', self.learning_rate
        
        # tf Graph input
        self.x = tf.placeholder(tf.float32, [None, network_architecture["n_input"]])
        self.keep_prob = tf.placeholder(tf.float32)
        
        self.h_dim = float(network_architecture["n_z"])
        self.h_dim_left = float(network_architecture["n_z_left"])
        self.a0 = np.ones((1 , 2)).astype(np.float32)/2
        self.a1 = np.ones((1 , self.h_dim_left)).astype(np.float32)/self.h_dim_left
        self.a2 = np.ones((1 , self.h_dim)).astype(np.float32)/self.h_dim
        
        self.mu2a0 = tf.constant((np.log(self.a0).T-np.mean(np.log(self.a0),1)).T)
        self.var2a0 = tf.constant(  ( ( (1.0/self.a0)*( 1 - (2.0/2.0) ) ).T +
                                ( 1.0/(self.h_dim*self.h_dim) )*np.sum(1.0/self.a0,1) ).T  )
        
        self.mu2a1 = tf.constant((np.log(self.a1).T-np.mean(np.log(self.a1),1)).T)
        self.var2a1 = tf.constant(  ( ( (1.0/self.a1)*( 1 - (2.0/self.h_dim_left) ) ).T +
                                ( 1.0/(self.h_dim_left*self.h_dim_left) )*np.sum(1.0/self.a1,1) ).T  )
        
        self.mu2a2 = tf.constant((np.log(self.a2).T-np.mean(np.log(self.a2),1)).T)
        self.var2a2 = tf.constant(  ( ( (1.0/self.a2)*( 1 - (2.0/self.h_dim) ) ).T +
                                ( 1.0/(self.h_dim*self.h_dim) )*np.sum(1.0/self.a2,1) ).T  )
        
        
        
        # Create autoencoder network
        self._create_network()
        # Define loss function based variational upper-bound and 
        # corresponding optimizer
        self._create_loss_optimizer()
        
        # Initializing the tensor flow variables
        init = tf.initialize_all_variables()

        # Launch the session
        self.sess = tf.InteractiveSession()
        self.sess.run(init)
    
    def _create_network(self):
        # Initialize autoencode network weights and biases
        self.network_weights = self._initialize_weights(**self.network_architecture)
        '''Adding 2 subtopics and 1 super topic.'''
        self.z_mean0,self.z_log_sigma_sq0 = \
            self._recognition_network(self.network_weights["weights_recog"], 
                                      self.network_weights["biases_recog"])
            
        self.z_mean1,self.z_log_sigma_sq1 = \
            self._recognition_network(self.network_weights["sub_01"], 
                                      self.network_weights["biases_sub_01"],1)
        
        self.z_mean2,self.z_log_sigma_sq2 = \
            self._recognition_network(self.network_weights["sub_01"], 
                                      self.network_weights["biases_sub_01"])

        # Draw one sample z from Gaussian distribution
        '''Adding another noise variable for super toipc: hard coding atm'''
        eps0 = tf.random_normal((1, 2), 0, 1, 
                               dtype=tf.float32)
        n_z = self.network_architecture["n_z"]
        n_z_left = self.network_architecture["n_z_left"]
        eps1 = tf.random_normal((1, n_z), 0, 1, 
                               dtype=tf.float32)
        eps2 = tf.random_normal((1, n_z_left), 0, 1, 
                               dtype=tf.float32)
        '''Adding RT for subtopics'''
        self.z0 = tf.nn.softmax(tf.contrib.layers.batch_norm(tf.add(self.z_mean0, 
                        tf.mul(tf.sqrt(tf.exp(self.z_log_sigma_sq0)), eps0))))
        self.z0 = tf.nn.softmax(tf.contrib.layers.batch_norm(self.z0))
        
        self.sigma0 = tf.exp(self.z_log_sigma_sq0)
        
        self.z1 = tf.add(self.z_mean1, 
                        tf.mul(tf.sqrt(tf.exp(self.z_log_sigma_sq1)), eps2))
        self.z1 = tf.nn.softmax(tf.contrib.layers.batch_norm(self.z1))
        
        self.sigma1 = tf.exp(self.z_log_sigma_sq1)
        
        self.z2 = tf.add(self.z_mean2, 
                        tf.mul(tf.sqrt(tf.exp(self.z_log_sigma_sq2)), eps1))
        self.z2 = tf.nn.softmax(tf.contrib.layers.batch_norm(self.z2))
        
        self.sigma2 = tf.exp(self.z_log_sigma_sq2)
        
        '''Adding subtopic reconstruction'''
            
#         self.theta = tf.matmul(self.z0,tf.concat(0,[self.z1, self.z2]))
       
            
        self.x_reconstr_mean_right = \
            self._generator_network(tf.nn.dropout(self.z2, self.keep_prob),
                                    self.network_weights["weights_gener"],
                                    self.network_weights["biases_gener"])
            
        self.x_reconstr_mean_left = \
            self._generator_network(tf.nn.dropout(self.z1, self.keep_prob),
                                    self.network_weights["weights_gener_left"],
                                    self.network_weights["biases_gener_left"])
            
    def _initialize_weights(self, n_hidden_recog_1, n_hidden_recog_2, 
                            n_hidden_gener_1,  
                            n_input, n_z,n_z_left):
        all_weights = dict()
        with tf.variable_scope("super"):
            all_weights['weights_recog'] = {
                'h1': tf.get_variable('h1',[n_input, n_hidden_recog_1]),
                'h2': tf.get_variable('h2',[n_hidden_recog_1, n_hidden_recog_2]),
                'out_mean': tf.get_variable('out_mean',[n_hidden_recog_2, 2]),
                'out_log_sigma': tf.get_variable('out_log_sigma',[n_hidden_recog_2, 2])
            }
            all_weights['biases_recog'] = {
                'b1': tf.Variable(tf.zeros([n_hidden_recog_1], dtype=tf.float32)),
                'b2': tf.Variable(tf.zeros([n_hidden_recog_2], dtype=tf.float32)),
                'out_mean': tf.Variable(tf.zeros([2], dtype=tf.float32)),
                'out_log_sigma': tf.Variable(tf.zeros([2], dtype=tf.float32))
            }
        with tf.variable_scope("sub01"):
            all_weights['sub_01'] = {
                'h1': tf.get_variable('h1',[n_input, n_hidden_recog_1]),
                'h2': tf.get_variable('h2',[n_hidden_recog_1, n_hidden_recog_2]),
                'out_mean': tf.get_variable('out_mean',[n_hidden_recog_2, n_z]),
                'out_log_sigma': tf.get_variable('out_log_sigma',[n_hidden_recog_2, n_z]),
                'out_mean_left': tf.get_variable('out_mean_left',[n_hidden_recog_2, n_z_left]),
                'out_log_sigma_left': tf.get_variable('out_log_sigma_left',[n_hidden_recog_2, n_z_left])}
            all_weights['biases_sub_01'] = {
                'b1': tf.Variable(tf.zeros([n_hidden_recog_1], dtype=tf.float32)),
                'b2': tf.Variable(tf.zeros([n_hidden_recog_2], dtype=tf.float32)),
                'out_mean': tf.Variable(tf.zeros([n_z], dtype=tf.float32)),
                'out_log_sigma': tf.Variable(tf.zeros([n_z], dtype=tf.float32)),
                'out_mean_left': tf.Variable(tf.zeros([n_z_left], dtype=tf.float32)),
                'out_log_sigma_left': tf.Variable(tf.zeros([n_z_left], dtype=tf.float32))
            }
        with tf.variable_scope("gen_left"):
            all_weights['weights_gener_left'] = {
                'h2': tf.Variable(xavier_init(n_z_left, n_hidden_gener_1))
            }
            all_weights['biases_gener_left'] = {
                'b2': tf.Variable(tf.zeros([n_hidden_gener_1], dtype=tf.float32))
            }
        with tf.variable_scope("gen"):
            all_weights['weights_gener'] = {
                'h2': tf.Variable(xavier_init(n_z, n_hidden_gener_1))
            }
            all_weights['biases_gener'] = {
                'b2': tf.Variable(tf.zeros([n_hidden_gener_1], dtype=tf.float32))
            }
        return all_weights
            
    def _recognition_network(self, weights, biases, switch=0):
        # Generate probabilistic encoder (recognition network)
        layer_1 = self.transfer_fct(tf.add(tf.matmul(self.x, weights['h1']), 
                                           biases['b1'])) 
        layer_2 = self.transfer_fct(tf.add(tf.matmul(layer_1, weights['h2']), 
                                           biases['b2'])) 
        layer_do = tf.nn.dropout(layer_2, self.keep_prob)
        if switch==0:
        
            z_mean = tf.contrib.layers.batch_norm(tf.add(tf.matmul(layer_do, weights['out_mean']),
                            biases['out_mean']))
            z_log_sigma_sq = \
                tf.contrib.layers.batch_norm(tf.add(tf.matmul(layer_do, weights['out_log_sigma']), 
                       biases['out_log_sigma']))    
        else:
            z_mean = tf.contrib.layers.batch_norm(tf.add(tf.matmul(layer_do, weights['out_mean_left']),
                            biases['out_mean_left']))
            z_log_sigma_sq = \
                tf.contrib.layers.batch_norm(tf.add(tf.matmul(layer_do, weights['out_log_sigma_left']), 
                       biases['out_log_sigma_left'])) 
        
        return (z_mean, z_log_sigma_sq)

    def _generator_network(self,z, weights, biases):
        # Generate probabilistic decoder (decoder network)
#         self.layer_do_0 = tf.nn.dropout(tf.nn.softmax(tf.contrib.layers.batch_norm(z)), self.keep_prob)
        self.layer_do_0 =z
        x_reconstr_mean = tf.add(tf.matmul(self.layer_do_0, 
                                           tf.nn.softmax(tf.contrib.layers.batch_norm(weights['h2']))),0.0)
#         x_reconstr_mean = tf.nn.softmax(tf.contrib.layers.batch_norm(tf.add(
#                     tf.matmul(self.layer_do_0, weights['h2']),0.0)))
    
        return x_reconstr_mean

            
    def _create_loss_optimizer(self):
        
        
        self.x_reconstr_mean  = tf.squeeze(tf.map_fn(lambda i: 
                                          tf.matmul(tf.expand_dims(tf.gather(self.z0,i),0),
                                                    tf.concat(0,[tf.expand_dims(tf.gather(self.x_reconstr_mean_left,i),0),
                                                                tf.expand_dims(tf.gather(self.x_reconstr_mean_right,i),0)])),
                                                   tf.constant(np.arange(self.batch_size)), dtype=tf.float32))
        
        self.x_reconstr_mean+=1e-10
     
        reconstr_loss = \
            -tf.reduce_sum(self.x * tf.log(self.x_reconstr_mean),1)#/tf.reduce_sum(self.x,1) 

    
        latent_loss0 = 0.5*( tf.reduce_sum(tf.div(self.sigma0,self.var2a0),1)+\
        tf.reduce_sum( tf.mul(tf.div((self.mu2a0 - self.z_mean0),self.var2a0),
                  (self.mu2a0 - self.z_mean0)),1) - 2 +\
                           tf.reduce_sum(tf.log(self.var2a0),1)  - tf.reduce_sum(self.z_log_sigma_sq0  ,1) ) 
        
        latent_loss1 = 0.5*( tf.reduce_sum(tf.div(self.sigma1,self.var2a1),1)+\
        tf.reduce_sum( tf.mul(tf.div((self.mu2a1 - self.z_mean1),self.var2a1),
                  (self.mu2a1 - self.z_mean1)),1) - self.h_dim_left +\
                           tf.reduce_sum(tf.log(self.var2a1),1)  - tf.reduce_sum(self.z_log_sigma_sq1  ,1) )
        
        latent_loss2 = 0.5*( tf.reduce_sum(tf.div(self.sigma2,self.var2a2),1)+\
        tf.reduce_sum( tf.mul(tf.div((self.mu2a2 - self.z_mean2),self.var2a2),
                  (self.mu2a2 - self.z_mean2)),1) - self.h_dim +\
                           tf.reduce_sum(tf.log(self.var2a2),1)  - tf.reduce_sum(self.z_log_sigma_sq2  ,1) )

        
        
#         latent_loss = -0.5 * tf.reduce_sum(1 + self.z_log_sigma_sq 
#                                            - tf.square(self.z_mean) 
#                                            - tf.exp(self.z_log_sigma_sq), 1)


        self.cost = tf.reduce_mean(reconstr_loss) \
    + tf.reduce_mean(latent_loss0 + latent_loss1 + latent_loss2) # average over batch
        
        # Use ADAM optimizer
        self.optimizer = \
            tf.train.AdamOptimizer(learning_rate=self.learning_rate,beta1=0.9).minimize(self.cost)
        
    def partial_fit(self, X):
        """Train model based on mini-batch of input data.
        
        Return cost of mini-batch.
        """
#         opt, cost,emb,kld = self.sess.run((self.optimizer, self.cost,self.network_weights['weights_gener']['h2'],
#                                           self.kld),feed_dict={self.x: X,self.keep_prob: .9})
        opt,cost,emb,emb2 = self.sess.run((self.optimizer, self.cost,self.network_weights['weights_gener']['h2'],
                                           self.network_weights['weights_gener_left']['h2']
                                      ),feed_dict={self.x: X,self.keep_prob: .7})
        return cost,emb,emb2
    
    def test(self, X):
        """Test the model and return the lowerbound on the log-likelihood.
        """
        cost,e1,e2 = self.sess.run((self.cost,self.z1,self.z2),
                                        feed_dict={self.x: np.expand_dims(X, axis=0),self.keep_prob: 1.0})
        return cost,e1,e2


In [6]:
tf.reset_default_graph()
cost_plot=[]
def train(network_architecture, learning_rate=0.0001,
          batch_size=100, training_epochs=10, display_step=5):
    vae = VariationalAutoencoder(network_architecture, 
                                 learning_rate=learning_rate, 
                                 batch_size=batch_size)
    emb=0
    avg_kld=0
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(n_samples_tr / batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs = minibatches.next()
            # Fit training using batch data
            cost,emb,emb2 = vae.partial_fit(batch_xs)
#             print cost
#             sys.exit()
            # Compute average loss
            avg_cost += cost / n_samples_tr * batch_size
            
            if np.isnan(avg_cost):
                print epoch,i,np.sum(batch_xs,1).astype(np.int),batch_xs.shape
                return vae,emb
#                 sys.exit()

        # Display logs per epoch step
        if epoch % display_step == 0:
            cost_plot.append(avg_cost)
            print "Epoch:", '%04d' % (epoch+1), \
                  "cost=", "{:.9f}".format(avg_cost)

    return vae,emb,emb2

In [7]:
tf.reset_default_graph()
network_architecture = \
    dict(n_hidden_recog_1=400, # 1st layer encoder neurons
         n_hidden_recog_2=400, # 2nd layer encoder neurons
         n_hidden_gener_1=docs_tr.shape[1], # 1st layer decoder neurons
         n_input=docs_tr.shape[1], # MNIST data input (img shape: 28*28)
         n_z=50,
         n_z_left=10)  # dimensionality of latent space

batch_size=200
learning_rate=0.005
def create_minibatch(data):
    rng = np.random.RandomState(10)
    
    while True:
        # Return random data samples of a size 'minibatch_size' at each iteration
        ixs = rng.randint(data.shape[0], size=batch_size)
        yield data[ixs]
       
minibatches = create_minibatch(docs_tr.astype('float32'))
start=time.time()
vae,emb,emb2 = train(network_architecture, training_epochs=700,batch_size=batch_size,learning_rate=learning_rate)
# plt.plot(cost_plot)
print 'it took',str(time.time()-start),'seconds'

'''
Change log: decoder has no dropout or additional tranformations
'''

/afs/inf.ed.ac.uk/group/cup/data1/akash_itm/IPC_MAP/lib/python2.7/site-packages/numpy/core/numeric.py:190: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  a = empty(shape, dtype, order)
/afs/inf.ed.ac.uk/group/cup/data1/akash_itm/IPC_MAP/lib/python2.7/site-packages/tensorflow/python/ops/gradients.py:90: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Learning Rate: 0.005
Epoch: 0001 cost= 12793.470703125
Epoch: 0006 cost= 11377.554296875
Epoch: 0011 cost= 10876.408593750
Epoch: 0016 cost= 10720.620442708
Epoch: 0021 cost= 10597.535156250
Epoch: 0026 cost= 10385.243880208
Epoch: 0031 cost= 10342.025390625
Epoch: 0036 cost= 10308.597395833
Epoch: 0041 cost= 10158.926953125
Epoch: 0046 cost= 10253.590885417
Epoch: 0051 cost= 10130.937239583
Epoch: 0056 cost= 10010.448567708
Epoch: 0061 cost= 10036.864583333
Epoch: 0066 cost= 9964.024218750
Epoch: 0071 cost= 9959.009505208
Epoch: 0076 cost= 10083.566536458
Epoch: 0081 cost= 10017.862630208
Epoch: 0086 cost= 9994.422395833
Epoch: 0091 cost= 9908.195572917
Epoch: 0096 cost= 9875.926692708
Epoch: 0101 cost= 9936.121744792
Epoch: 0106 cost= 9972.556119792
Epoch: 0111 cost= 9878.376822917
Epoch: 0116 cost= 9890.229166667
Epoch: 0121 cost= 9847.964062500
Epoch: 0126 cost= 9825.926953125
Epoch: 0131 cost= 9886.133723958
Epoch: 0136 cost= 9816.139583333
Epoch: 0141 cost= 9824.579557292
Epoch: 

'\nChange log: decoder has no dropout or additional tranformations\n'

In [9]:
def print_top_words(beta, feature_names, n_top_words=5):
    for i in range(len(beta)):
        print(" ".join([feature_names[j]
            for j in beta[i].argsort()[:-n_top_words - 1:-1]]))

print_top_words(emb2, "".join(vocab).split('\n'))

transistor vlsi analog cmos chip
limit macroscopic phy ansatz states
sirosh kenneth stdev bishop settled
learning potential mushroom linearized glomerular
auditory sound sparrow gene birdsong
command reward predisposition barto chick
incremented network pacemaker clocking tokyo
rgbn nonlocal rni clipped unsmearing
image images isard view pentland
regional explosion anomalous eeg nuclear


In [10]:

cost=[]
e1=[]
e2=[]
idx=0
for doc in docs_te:
    idx+=1
    doc = doc.astype('float32')
    n_d = np.sum(doc)
    c,a,b=vae.test(doc)
    cost.append(c/n_d)
    e1.append(a)
    e2.append(b)
    if idx%1000==0:
        print idx
    
# print (np.exp(np.mean(np.array(cost))))
# print np.mean(e1,0)
# print np.mean(e2,0)

1000
2000
3000
4000
5000
6000
7000


In [57]:
print np.argsort(np.mean(e1,0)[0])
print np.argsort(np.mean(e2,0)[0])
print np.mean(e1,0)#[0]
print np.mean(e2,0)#[0]
print np.exp(np.mean(np.array(cost)))

[4 7 1 2 9 3 0 6 5 8]
[ 5 36 31 49 30 15 18 29 10 42 12 14 38 22 21  2 48 11 46  7 37  6 47  1 34
 26 32 25 13  4 24 27 33 45 20  0 43 41 44 23 19 35 40 39 16  3 17 28  8  9]
[[ 0.10906594  0.07742675  0.07825247  0.10420682  0.03602751  0.15684767
   0.11375484  0.05458829  0.16575989  0.10410608]]
[[ 0.02128627  0.01403267  0.01166745  0.04748262  0.01918073  0.00052926
   0.01365676  0.01269583  0.05084285  0.05804456  0.00602656  0.01225295
   0.00770669  0.01899555  0.00789725  0.00525684  0.04729966  0.04854282
   0.00530473  0.03680063  0.02126016  0.01148158  0.00981647  0.0360223
   0.01954816  0.01875053  0.01553394  0.02018736  0.05030089  0.00593949
   0.00524464  0.00335969  0.01723636  0.0205913   0.01418491  0.03688427
   0.00100458  0.01364144  0.00809556  0.0387565   0.03848539  0.02482721
   0.00666456  0.02290773  0.03118976  0.02097143  0.01245169  0.01378405
   0.01166757  0.00370748]]
1.06634e+10


In [19]:
pickle.dump(cost,open('cost','w'))
pickle.dump(e1,open('e1','w'))
pickle.dump(e2,open('e2','w'))

In [1]:
e=e2
print np.amax(np.mean(e,0))
print np.min(np.mean(e,0))

NameError: name 'e2' is not defined